In [49]:
class BSTNode:
    def __init__(self, time_str, destination):
        self.time_str = time_str
        self.destination = destination
        self.left = None
        self.right = None


class BSTNode:
    def __init__(self, time_str, destination):
        self.time_str = time_str
        self.destination = destination
        self.left = None
        self.right = None


class BST:
    def __init__(self):
        self.root = None

    def insert(self, time_str, destination):
        self.root = self._insert(self.root, time_str, destination)

    def _insert(self, node, time_str, destination):
        if node is None:
            return BSTNode(time_str, destination)
        
        # Check if the time_str is equal to an existing node's time_str
        if time_str < node.time_str:
            node.left = self._insert(node.left, time_str, destination)
        elif time_str > node.time_str:
            node.right = self._insert(node.right, time_str, destination)
        else:  # time_str == node.time_str, we insert a new node with the same time
            node.left = self._insert(node.left, time_str, destination)
        
        return node

    def find(self, time_str):
        return self._find(self.root, time_str)

    def _find(self, node, time_str):
        if node is None:
            return None
        
        if time_str == node.time_str:
            return node
        elif time_str < node.time_str:
            return self._find(node.left, time_str)
        else:
            return self._find(node.right, time_str)

    def delete(self, time_str):
        self.root, deleted_node = self._delete(self.root, time_str)
        return deleted_node

    def _delete(self, node, time_str):
        if node is None:
            return node, None

        if time_str < node.time_str:
            node.left, deleted_node = self._delete(node.left, time_str)
        elif time_str > node.time_str:
            node.right, deleted_node = self._delete(node.right, time_str)
        else:
            deleted_node = node
            if node.left is None:
                return node.right, deleted_node
            elif node.right is None:
                return node.left, deleted_node
            else:
                min_node = self._find_min(node.right)
                node.time_str, node.destination = min_node.time_str, min_node.destination
                node.right, _ = self._delete(node.right, min_node.time_str)

        return node, deleted_node

    def _find_min(self, node):
        while node.left is not None:
            node = node.left
        return node

    def destination(self, time_str):
        node = self.find(time_str)
        if node is None:
            return "-"
        return node.destination

    def cancel(self, time_str):
        deleted_node = self.delete(time_str)
        if deleted_node is None:
            return "-"
        return f"Flight at {time_str} canceled"

    def delay(self, time_str, seconds):
        node = self.find(time_str)
        if node is None:
            return "-"
        new_time = self._add_seconds(time_str, seconds)
        self.delete(time_str)
        self.insert(new_time, node.destination)
        return f"Flight at {time_str} delayed to {new_time}"

    def reroute(self, time_str, new_destination):
        node = self.find(time_str)
        if node is None:
            return "-"
        node.destination = new_destination
        return f"Flight at {time_str} rerouted to {new_destination}"

    def next_departure(self, time_str):
        return self._next_departure(self.root, time_str)

    def _next_departure(self, node, time_str):
        if node is None:
            return None

        if node.time_str >= time_str:
            left_result = self._next_departure(node.left, time_str)
            if left_result:
                return left_result
            return node.time_str, node.destination

        return self._next_departure(node.right, time_str)

    def count_in_range(self, start, end):
        return self._count_in_range(self.root, start, end)

    def _count_in_range(self, node, start, end):
        if node is None:
            return 0

        count = 0
        if start <= node.time_str <= end:
            count = 1

        count += self._count_in_range(node.left, start, end)
        count += self._count_in_range(node.right, start, end)

        return count

    def _add_seconds(self, time_str, seconds):
        from datetime import datetime, timedelta
        time = datetime.strptime(time_str, "%H:%M:%S")
        new_time = time + timedelta(seconds=seconds)
        return new_time.strftime("%H:%M:%S")


In [38]:
import sys
from io import StringIO

# Simulate input as if it were from stdin
input_data = """10 10
09:00:00 Chicago
09:01:00 Phoenix
09:02:00 Houston
09:03:00 Seattle
09:04:00 Miami
09:05:00 Dallas
09:06:00 Boston
09:07:00 NewYork
09:08:00 Austin
09:09:00 Denver
destination 09:01:00
destination 09:03:00
cancel 09:02:00
destination 09:02:00
delay 09:03:00 60
destination 09:04:00
reroute 09:05:00 Tokyo
destination 09:05:00
next 09:05:30
count 09:00:00 09:06:00
"""

In [50]:

sys.stdin = StringIO(input_data)
# Create BST instance
bst = BST()

# Read all input at once
lines = sys.stdin.read().splitlines()
#print(lines)
n, q = map(int, lines[0].split())

# Read and insert n flights
for i in range(1, n + 1):
    time_str, dest = lines[i].split()
    bst.insert(time_str, dest)

# Process q queries
for i in range(n + 1, n + q + 1):
    parts = lines[i].split()
    cmd = parts[0]

    if cmd == "destination":
        time_str = parts[1]
        print(bst.destination(time_str))

    elif cmd == "cancel":
        time_str = parts[1]
        bst.cancel(time_str)

    elif cmd == "delay":
        time_str = parts[1]
        seconds = int(parts[2])
        bst.delay(time_str, seconds)

    elif cmd == "reroute":
        time_str = parts[1]
        new_dest = parts[2]
        bst.reroute(time_str, new_dest)

    elif cmd == "next":
        result = bst.next_departure(parts[1])
        if result == "not found":
            print("-")
        else:
            time, dest = result
            print(f"{time} {dest}")

    elif cmd == "count":
        start, end = parts[1], parts[2]
        print(bst.count_in_range(start, end))


Phoenix
Seattle
-
Miami
Tokyo
09:06:00 Boston
6


In [ ]:
import sys

# Create BST instance (make sure the BST class is defined as needed)
bst = BST()

# Read all input at once
lines = sys.stdin.read().splitlines()

# First line: Read n (number of flights) and q (number of queries)
n, q = map(int, lines[0].split())

# Read and insert n flights
for i in range(1, n + 1):
    time_str, dest = lines[i].split()
    bst.insert(time_str, dest)

# Process q queries
for i in range(n + 1, n + q + 1):
    parts = lines[i].split()
    cmd = parts[0]

    if cmd == "destination":
        time_str = parts[1]
        print(bst.destination(time_str))

    elif cmd == "cancel":
        time_str = parts[1]
        bst.cancel(time_str)

    elif cmd == "delay":
        time_str = parts[1]
        seconds = int(parts[2])
        bst.delay(time_str, seconds)

    elif cmd == "reroute":
        time_str = parts[1]
        new_dest = parts[2]
        bst.reroute(time_str, new_dest)

    elif cmd == "next":
        result = bst.next_departure(parts[1])
        if result == "not found":
            print("-")
        else:
            time, dest = result
            print(f"{time} {dest}")

    elif cmd == "count":
        start, end = parts[1], parts[2]
        print(bst.count_in_range(start, end))


In [53]:
import sys
from io import StringIO
import bisect
from datetime import datetime, timedelta

def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

def seconds_to_time(seconds):
    h = seconds // 3600
    seconds %= 3600
    m = seconds // 60
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

# Example input data as a list of strings
input_data = """14 10
09:00:00 Chicago
09:00:03 Phoenix
09:00:13 Houston
09:00:59 Chicago
09:01:10 Houston
09:03:13 Chicago
09:10:11 Seattle
09:10:25 Seattle
09:14:25 Phoenix
09:19:32 Chicago
09:19:46 Chicago
09:21:05 Chicago
09:22:43 Seattle
09:22:54 Seattle
destination 09:00:03
destination 09:21:45
destination 09:14:25
cancel 09:14:25
destination 09:14:25
delay 09:00:59 1
destination 09:00:59
destination 09:01:00
reroute 09:01:00 Houston
destination 09:01:00"""

# Redirect standard input to read from the input_data string
sys.stdin = StringIO(input_data)

# Read all lines from the redirected stdin
lines = sys.stdin.read().splitlines()

n, m = map(int, lines[0].split())
flights = []
time_to_dest = {}
times_in_seconds = []

for i in range(1, n + 1):
    time_str, dest = lines[i].split()
    seconds = time_to_seconds(time_str)
    flights.append((seconds, dest))
    time_to_dest[seconds] = dest
    times_in_seconds.append(seconds)

# We need to keep times_in_seconds sorted for bisect operations (it is initially sorted)
output = []

for i in range(n + 1, n + m + 1):
    parts = lines[i].split()
    op = parts[0]
    
    if op == "destination":
        t = parts[1]
        t_sec = time_to_seconds(t)
        if t_sec in time_to_dest:
            output.append(time_to_dest[t_sec])
        else:
            output.append("-")
    
    elif op == "cancel":
        s = parts[1]
        s_sec = time_to_seconds(s)
        if s_sec in time_to_dest:
            del time_to_dest[s_sec]
            index = bisect.bisect_left(times_in_seconds, s_sec)
            if index < len(times_in_seconds) and times_in_seconds[index] == s_sec:
                times_in_seconds.pop(index)
    
    elif op == "delay":
        s = parts[1]
        d = int(parts[2])
        s_sec = time_to_seconds(s)
        if s_sec in time_to_dest:
            dest = time_to_dest[s_sec]
            new_s_sec = s_sec + d
            new_s_str = seconds_to_time(new_s_sec)
            # Remove old entries
            del time_to_dest[s_sec]
            index = bisect.bisect_left(times_in_seconds, s_sec)
            if index < len(times_in_seconds) and times_in_seconds[index] == s_sec:
                times_in_seconds.pop(index)
            # Add new entries
            bisect.insort(times_in_seconds, new_s_sec)
            time_to_dest[new_s_sec] = dest
    
    elif op == "reroute":
        s = parts[1]
        c = parts[2]
        s_sec = time_to_seconds(s)
        if s_sec in time_to_dest:
            time_to_dest[s_sec] = c
    
    elif op == "next":
        t = parts[1]
        t_sec = time_to_seconds(t)
        # Find the first s >= t_sec in times_in_seconds
        index = bisect.bisect_left(times_in_seconds, t_sec)
        if index < len(times_in_seconds):
            s_sec = times_in_seconds[index]
            output.append(seconds_to_time(s_sec))
        else:
            pass  # per problem statement, it's guaranteed to exist
    
    elif op == "count":
        t1 = parts[1]
        t2 = parts[2]
        t1_sec = time_to_seconds(t1)
        t2_sec = time_to_seconds(t2)
        left = bisect.bisect_left(times_in_seconds, t1_sec)
        right = bisect.bisect_right(times_in_seconds, t2_sec)
        output.append(str(right - left))

print('\n'.join(output))

Phoenix
-
Phoenix
-
-
Chicago
Houston


In [ ]:
import sys
import bisect # Binary search functions

def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

def seconds_to_time(seconds):
    h = seconds // 3600
    seconds %= 3600
    m = seconds // 60
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

def main():
    # Read all input lines
    lines = sys.stdin.read().splitlines()
    n, m = map(int, lines[0].split())
    time_to_dest = {}
    times_in_seconds = []

    for i in range(1, n + 1):
        time_str, dest = lines[i].split()
        seconds = time_to_seconds(time_str)
        time_to_dest[seconds] = dest
        times_in_seconds.append(seconds)

    output = []

    for i in range(n + 1, n + m + 1):
        #Current line split into 2
        parts = lines[i].split()
        #Current operation
        op = parts[0]
        
        if op == "destination":
            #gets the time in the second part
            t = parts[1]
            #converts it to seconds
            t_sec = time_to_seconds(t)
            #Checks if its in the dictionary
            if t_sec in time_to_dest:
                #if it is, then append value to our output
                output.append(time_to_dest[t_sec])
            else:
                #if not, then just - as decribed in the kattis
                output.append("-")
        
        elif op == "cancel":
            #Check seconds.
            s = parts[1]
            s_sec = time_to_seconds(s)
            if s_sec in time_to_dest:
                del time_to_dest[s_sec]
                index = bisect.bisect_left(times_in_seconds, s_sec)
                if index < len(times_in_seconds) and times_in_seconds[index] == s_sec:
                    times_in_seconds.pop(index)
        
        elif op == "delay":
            #Get the timestamp
            s = parts[1]
            #Get the delayed seconds
            d = int(parts[2])
            s_sec = time_to_seconds(s)
            if s_sec in time_to_dest:
                dest = time_to_dest[s_sec]
                new_s_sec = s_sec + d
                # Remove old entries
                del time_to_dest[s_sec]
                index = bisect.bisect_left(times_in_seconds, s_sec)
                if index < len(times_in_seconds) and times_in_seconds[index] == s_sec:
                    times_in_seconds.pop(index)
                # Add new entries
                bisect.insort(times_in_seconds, new_s_sec)
                time_to_dest[new_s_sec] = dest
        
        elif op == "reroute":
            s = parts[1]
            c = parts[2]
            s_sec = time_to_seconds(s)
            if s_sec in time_to_dest:
                time_to_dest[s_sec] = c
        
        elif op == "next":
            t = parts[1]
            t_sec = time_to_seconds(t)
            # Find the first s >= t_sec in times_in_seconds
            index = bisect.bisect_left(times_in_seconds, t_sec)
            if index < len(times_in_seconds):
                s_sec = times_in_seconds[index]
                dest = time_to_dest[s_sec]
                output.append(f"{seconds_to_time(s_sec)} {dest}")
            else:
                pass  # per problem statement, it's guaranteed to exist
        
        elif op == "count":
            t1 = parts[1]
            t2 = parts[2]
            t1_sec = time_to_seconds(t1)
            t2_sec = time_to_seconds(t2)
            left = bisect.bisect_left(times_in_seconds, t1_sec)
            right = bisect.bisect_right(times_in_seconds, t2_sec)
            output.append(str(right - left))

    print('\n'.join(output))

if __name__ == "__main__":
    main()